In [1]:
import os
import pickle as pkl
import spacy
import csv
import json
from copy import deepcopy
from tqdm import tqdm
# import benepar
# import re
from string import punctuation

# from utils import merge_maximum_span
# from utils import clean_sentence_brackets
# from utils import process_nps_punctuation
# from utils import extract_scenes

In [20]:
def combine_multiple_scenes(scenes):
    # Store Data
    sentences = []
    querySpans = []
    candidateSpans = []
    clickSpans = []
    sentence_offsets = [0]
    querySpans_offsets = [0]
    option1 = []
    option2 = []
    scene_ids = []

    for instance in scenes:
        offset = len(sentences)
        sentences.extend(instance['sentences'])
        op1 = deepcopy(instance['option1'])
        for token in op1:
            for query in token['mentions']:
                query['sentenceIndex'] = str(int(query['sentenceIndex']) + offset)
        option1.extend(op1)

        op2 = deepcopy(instance['option2'])
        for token in op2:
            for query in token['mentions']:
                query['sentenceIndex'] = str(int(query['sentenceIndex']) + offset)
        option2.extend(op2)
        for item in instance['querySpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            querySpans.append(token)
        for item in instance['candidateSpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            candidateSpans.append(token)
        for item in instance['clickSpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            clickSpans.append(token)
        sentence_offsets.append(len(sentences))
        querySpans_offsets.append(len(querySpans))
        scene_ids.extend(instance['scene_ids'])
    return {
    "sentences": sentences,
    "querySpans": querySpans,
    "option1": option1,
    "option2": option2,
    "candidateSpans": candidateSpans,
    "clickSpans": clickSpans,
    "sentence_offsets": sentence_offsets,
    "querySpans_offsets": querySpans_offsets,
    "scene_ids": scene_ids,
    }

In [23]:
# Generate Adjudication File
input_path = "adjudication.pkl"
output_path = "adjudication_input/adjudication.csv"

with open(input_path, 'rb') as f:
    data = pkl.load(f)
    print(len(data))
    # Batch Scenes
    output = []
    for i in range(len(data)//5+1):
        output.append(combine_multiple_scenes(data[i*5: i*5+5]))

    with open(output_path, 'w', encoding='utf-8') as csv_fh:
        fieldnames = ['json_data']
        writer = csv.DictWriter(csv_fh, fieldnames, lineterminator='\n')
        writer.writeheader()
        for line in output:
            writer.writerow({'json_data': json.dumps(line)})

268


In [24]:
count = 0
for item in output:
    # print(len(item['querySpans']))
    count += len(item['querySpans'])
print(count)

3146


In [171]:
sample = output[0]
for item in sample:
    print(item)
    print(sample[item])
    print()

sentences
[['Leonard', ':', 'Sheldon', ',', 'it', "'s", 'not', 'a', 'great', 'time', '!'], ['Sheldon', ':', 'I', "'m", 'sorry', ',', 'but', 'this', 'is', 'important', '.'], ['Leonard', ':', 'What', 'is', 'it', '?'], ['Sheldon', ':', 'Back', 'to', 'the', 'Future', 'II', 'was', 'in', 'the', 'Back', 'to', 'the', 'Future', 'III', 'case', '...', 'and', 'Back', 'to', 'the', 'Future', 'III', 'was--', 'get', 'this--', 'in', 'the', 'Back', 'to', 'the', 'Future', 'II', 'case', '.'], ['Leonard', ':', 'So', '?'], ['Sheldon', ':', 'So', ',', 'did', 'you', 'do', 'that', ',', 'or', 'am', 'I', 'in', 'the', 'house', 'with', 'an', 'intruder', '?'], ['Leonard', ':', 'Sheldon', ',', 'I', 'got', 'to', 'go', 'inside', '.', 'It', "'s", 'getting', 'rough', 'out', 'here', '.'], ['Sheldon', ':', 'You', "'re", 'dodging', 'the', 'question', ';', 'I', 'knew', 'it', 'was', 'you', '.'], ['Leonard', ':', 'What', 'was', 'what', '?'], ['Sheldon', ':', 'This', 'is', "n't", 'a', 'very', 'good', 'connection', ',', 'but', 